# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [1]:
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
df_5_rows = pd.read_csv("https://bit.ly/3H2XVgC", nrows = 5)
df_5_rows
#print(df_5_rows.isna().sum())
#print(df_5_rows.info())


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [2]:
#df_full = pd.read_csv("https://bit.ly/3H2XVgC")
#df_full.describe()

In [3]:
df_5000_rows = pd.read_csv("https://bit.ly/3H2XVgC", nrows = 10000)
print(df_5000_rows.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 52 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          10000 non-null  int64  
 1   member_id                   10000 non-null  float64
 2   loan_amnt                   10000 non-null  float64
 3   funded_amnt                 10000 non-null  float64
 4   funded_amnt_inv             10000 non-null  float64
 5   term                        10000 non-null  object 
 6   int_rate                    10000 non-null  object 
 7   installment                 10000 non-null  float64
 8   grade                       10000 non-null  object 
 9   sub_grade                   10000 non-null  object 
 10  emp_title                   9348 non-null   object 
 11  emp_length                  9645 non-null   object 
 12  home_ownership              10000 non-null  object 
 13  annual_inc                  1000

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [4]:
# Your code goes here
import numpy as np
num_type_cols = list()
str_type_cols = list()
chunk_counter = 1
memory_footprints = []

numerics = ['int8','int16','int32','int64','float16','float32','float64','float128']
chunk_iter = pd.read_csv("https://bit.ly/3H2XVgC", chunksize = 3000)
for chunk in chunk_iter:
  num_type_cols = chunk.select_dtypes(include=numerics).columns
  #num_type_cols = chunk.select_dtypes(include=np.number).columns
  str_type_cols = chunk.select_dtypes(include=['object']).columns

  print(f"Chunk {chunk_counter} details")
  print(f"\tNumeric Type Columns = {len(num_type_cols)}")
  print(f"\tString Type Columns = {len(str_type_cols)}")

  str_col_uniq_values = {}
  str_col_uniq_lt_50 = list()
  for col in str_type_cols:
    num_unique_values = len(chunk[col].unique())
    str_col_uniq_values[col] = num_unique_values

    num_total_values = len(chunk[col])
    if num_unique_values / num_total_values < 0.5:
      str_col_uniq_lt_50.append(col)

  print(f'\tUnique values per string column: {str_col_uniq_values}')
  print(f'\tString columns with < 50% unique values: {str_col_uniq_lt_50}')
    
  float_chunk = chunk.select_dtypes(include=['float16','float32','float64','float128'])
  float_null_count = float_chunk.isnull().sum()
  float_cols_no_nulls = [col for col in float_chunk.columns if float_chunk[col].isnull().sum() == 0]
  #print(f'\t{float_chunk.isnull().sum()}')

  chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
  memory_footprints.append(chunk_memory)
  
  print(f'\tFloat columns with no missing values: {float_cols_no_nulls}')
  print(f'\tChunk Memory: {chunk_memory} MB')
  print("\n\n")
  chunk_counter += 1
print(f'\tTotal Memory: {sum(memory_footprints)}')

Chunk 1 details
	Numeric Type Columns = 31
	String Type Columns = 21
	Unique values per string column: {'term': 2, 'int_rate': 36, 'grade': 7, 'sub_grade': 35, 'emp_title': 2654, 'emp_length': 12, 'home_ownership': 3, 'verification_status': 3, 'issue_d': 2, 'loan_status': 6, 'pymnt_plan': 1, 'purpose': 13, 'title': 1406, 'zip_code': 568, 'addr_state': 43, 'earliest_cr_line': 366, 'revol_util': 884, 'initial_list_status': 1, 'last_pymnt_d': 55, 'last_credit_pull_d': 55, 'application_type': 1}
	String columns with < 50% unique values: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type']
	Float columns with no missing values: ['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6

## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [5]:
# Your code goes here
# I will convert columns int_rate and revol_util to numeric because by removing the % sign at the end.
num_type_cols = list()
str_type_cols = list()
chunk_counter = 1
memory_footprints = []

numerics = ['int8','int16','int32','int64','float16','float32','float64','float128']
chunk_iter = pd.read_csv("https://bit.ly/3H2XVgC", chunksize = 3000)
for chunk in chunk_iter:
  chunk['int_rate'] = chunk['int_rate'].str.rstrip('%').astype('float') / 100
  chunk['revol_util'] = chunk['revol_util'].str.rstrip('%').astype('float') / 100
  
  num_type_cols = chunk.select_dtypes(include=numerics).columns
  #num_type_cols = chunk.select_dtypes(include=np.number).columns
  str_type_cols = chunk.select_dtypes(include=['object']).columns

  print(f"Chunk {chunk_counter} details")
  print(f"\tNumeric Type Columns = {len(num_type_cols)}")
  print(f"\tString Type Columns = {len(str_type_cols)}")

  str_col_uniq_values = {}
  str_col_uniq_lt_50 = list()
  for col in str_type_cols:
    num_unique_values = len(chunk[col].unique())
    str_col_uniq_values[col] = num_unique_values

    num_total_values = len(chunk[col])
    if num_unique_values / num_total_values < 0.5:
      str_col_uniq_lt_50.append(col)

  for col in str_col_uniq_lt_50:
    chunk[col] = chunk[col].astype('category')
    
  float_chunk = chunk.select_dtypes(include=['float16','float32','float64','float128'])
  float_null_count = float_chunk.isnull().sum()
  float_cols_no_nulls = [col for col in float_chunk.columns if float_chunk[col].isnull().sum() == 0]
  #print(f'\t{float_chunk.isnull().sum()}')

  chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
  memory_footprints.append(chunk_memory)

  print(f'\tFloat columns with no missing values: {float_cols_no_nulls}')
  print(f'\tChunk Memory: {chunk_memory} MB')
  print("\n\n")
  chunk_counter += 1
print(f'\tTotal Memory: {sum(memory_footprints)}')

Chunk 1 details
	Numeric Type Columns = 33
	String Type Columns = 19
	Float columns with no missing values: ['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med', 'policy_code', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']
	Chunk Memory: 1.2565689086914062 MB



Chunk 2 details
	Numeric Type Columns = 33
	String Type Columns = 19
	Float columns with no missing values: ['member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc', 'out_prncp', 'out_prncp

## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [6]:
# Your code goes here
# I will convert columns int_rate and revol_util to numeric because by removing the % sign at the end.
num_type_cols = list()
str_type_cols = list()
chunk_counter = 1
memory_footprints = []

#By inspection, the following float columns can be converted to integer types: 
float_to_int_cols = ['member_id','loan_amnt','funded_amnt','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_bal','total_acc','policy_code','acc_now_delinq','pub_rec_bankruptcies']

numerics = ['int8','int16','int32','int64','float16','float32','float64','float128']
chunk_iter = pd.read_csv("https://bit.ly/3H2XVgC", chunksize = 3000)
for chunk in chunk_iter:
  chunk['int_rate'] = chunk['int_rate'].str.rstrip('%').astype('float') / 100
  chunk['revol_util'] = chunk['revol_util'].str.rstrip('%').astype('float') / 100
  
  num_type_cols = chunk.select_dtypes(include=numerics).columns
  #num_type_cols = chunk.select_dtypes(include=np.number).columns
  str_type_cols = chunk.select_dtypes(include=['object']).columns

  print(f"Chunk {chunk_counter} details")

  str_col_uniq_values = {}
  str_col_uniq_lt_50 = list()
  for col in str_type_cols:
    num_unique_values = len(chunk[col].unique())
    str_col_uniq_values[col] = num_unique_values

    num_total_values = len(chunk[col])
    if num_unique_values / num_total_values < 0.5:
      str_col_uniq_lt_50.append(col)

  for col in str_col_uniq_lt_50:
    chunk[col] = chunk[col].astype('category')
    
  float_chunk = chunk.select_dtypes(include=['float16','float32','float64','float128'])
  float_null_count = float_chunk.isnull().sum()
  float_cols_no_nulls = [col for col in float_chunk.columns if float_chunk[col].isnull().sum() == 0]
  float_cols_nulls = [col for col in float_chunk.columns if float_chunk[col].isnull().sum() > 0]

  for col in float_cols_nulls:
    chunk[col] = pd.to_numeric(chunk[col], downcast='float')

  # Convert selected float columns to integer per chunk
  for col_name in float_cols_no_nulls:
    if col_name in float_to_int_cols:
      col_max = chunk[col_name].max()
      col_min = chunk[col_name].min()
      for dtype_name in ['int8', 'int16', 'int32', 'int64']:
        if col_max <  np.iinfo(dtype_name).max and col_min > np.iinfo(dtype_name).min:
          chunk[col_name] = chunk[col_name].astype(dtype_name)
          break
  #print(f'\t{float_chunk.isnull().sum()}')
  chunk_memory = chunk.memory_usage(deep=True).sum()/(1024*1024)
  memory_footprints.append(chunk_memory)
  
  print(f'\tChunk Memory: {chunk_memory} MB')
  #print(chunk.info())
  print("\n")
  chunk_counter += 1
print(f'\tTotal Memory: {sum(memory_footprints)}')

Chunk 1 details
	Chunk Memory: 1.0505752563476562 MB


Chunk 2 details
	Chunk Memory: 1.0505170822143555 MB


Chunk 3 details
	Chunk Memory: 1.0650091171264648 MB


Chunk 4 details
	Chunk Memory: 1.1137962341308594 MB


Chunk 5 details
	Chunk Memory: 1.1119632720947266 MB


Chunk 6 details
	Chunk Memory: 1.1242618560791016 MB


Chunk 7 details
	Chunk Memory: 1.1113080978393555 MB


Chunk 8 details
	Chunk Memory: 1.1021366119384766 MB


Chunk 9 details
	Chunk Memory: 1.1001548767089844 MB


Chunk 10 details
	Chunk Memory: 1.0989570617675781 MB


Chunk 11 details
	Chunk Memory: 1.1115789413452148 MB


Chunk 12 details
	Chunk Memory: 1.1203765869140625 MB


Chunk 13 details
	Chunk Memory: 1.1322851181030273 MB


Chunk 14 details
	Chunk Memory: 1.1462459564208984 MB


Chunk 15 details
	Chunk Memory: 0.25269317626953125 MB


	Total Memory: 15.691859245300293


## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [7]:
# Your code goes here